In [6]:
import FlowCal
import pandas as pd
import numpy as np
import os
import matplotlib as plt
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
# from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.metrics import classification_report, confusion_matrix

In [7]:
labels = pd.read_excel('EU_label.xlsx')
channel_mapping = pd.read_excel('EU_marker_channel_mapping.xlsx')
use_channels = channel_mapping[channel_mapping['use'] == 1]
use_channels = use_channels['PxN(channel)']
use_channels

0                 SSC-W
1                 SSC-A
2                 SSC-H
3                 FSC-W
4                 FSC-A
5                 FSC-H
7       FJComp-BUV737-A
8          FJComp-APC-A
9        FJComp-BV711-A
10     FJComp-BB700-P-A
11       FJComp-BB630-A
12      FJComp-BUV395-A
13      FJComp-BUV563-A
14       FJComp-BV480-A
15       FJComp-BV421-A
16       FJComp-BV650-A
17      FJComp-BYG584-A
18    FJComp-PE-CF594-A
19    FJComp-BUV615-P-A
20      FJComp-BUV805-A
21      FJComp-BYG790-A
22    FJComp-PE-Cy5.5-A
23       FJComp-BV570-A
24      FJComp-BUV496-A
25       FJComp-BV605-A
27       FJComp-BV786-A
28    FJComp-APC-R700-A
30      FJComp-BYG670-A
31     FJComp-BV750-P-A
32        FJComp-FITC-A
33      FJComp-APC-H7-A
Name: PxN(channel), dtype: object

In [8]:
for dirpath, dirnames, filenames in os.walk('raw_fcs'):
    for filename in filenames:
        if filename.endswith('.fcs'):
            full_path = os.path.join(dirpath, filename)
            parent_dir = os.path.dirname(full_path)
            parent_dirname = os.path.basename(parent_dir)
            new_filename = parent_dirname + '.fcs'
            new_filepath = os.path.join(parent_dir, new_filename)
            os.rename(full_path, new_filepath)

In [9]:
data_dict = {}
for dirpath, dirnames, filenames in os.walk('raw_fcs'):
    for filename in filenames:
        if filename.endswith('.fcs'):
            full_path = os.path.join(dirpath, filename)
            data = FlowCal.io.FCSData(full_path)
            data = FlowCal.transform.to_rfi(data, channels=use_channels)
            data_case = {channel: data[:, channel] for channel in use_channels}
            case_id = os.path.splitext(filename)[0]
            data_dict[case_id] = data_case

In [10]:
scalers = {}

normalized_data_dict = {}

for sample, channels in data_dict.items():
    normalized_data_dict[sample] = {}
    scalers[sample] = {}

    for channel, array in channels.items():
        scaler = StandardScaler()
        
        scaler.fit(array.reshape(-1, 1))
        
        normalized_array = scaler.transform(array.reshape(-1, 1))
        
        scalers[sample][channel] = scaler
        
        normalized_data_dict[sample][channel] = normalized_array

In [11]:
df = pd.read_excel('EU_label.xlsx')
labels_dict = {}
for index, row in df.iterrows():
    case_id = row['file_flow_id']
    health_status = row['label']
    label = 0 if health_status == 'Healthy' else 1
    labels_dict[case_id] = label

In [12]:
for case_id in normalized_data_dict.keys():
    print(case_id, len(normalized_data_dict[case_id]))

flowrepo_covid_EU_002_flow_001 31
flowrepo_covid_EU_003_flow_001 31
flowrepo_covid_EU_004_flow_001 31
flowrepo_covid_EU_005_flow_001 31
flowrepo_covid_EU_006_flow_001 31
flowrepo_covid_EU_007_flow_001 31
flowrepo_covid_EU_008_flow_001 31
flowrepo_covid_EU_009_flow_001 31
flowrepo_covid_EU_010_flow_001 31
flowrepo_covid_EU_011_flow_001 31
flowrepo_covid_EU_012_flow_001 31
flowrepo_covid_EU_013_flow_001 31
flowrepo_covid_EU_014_flow_001 31
flowrepo_covid_EU_015_flow_001 31
flowrepo_covid_EU_016_flow_001 31
flowrepo_covid_EU_017_flow_001 31
flowrepo_covid_EU_018_flow_001 31
flowrepo_covid_EU_019_flow_001 31
flowrepo_covid_EU_020_flow_001 31
flowrepo_covid_EU_021_flow_001 31
flowrepo_covid_EU_022_flow_001 31
flowrepo_covid_EU_023_flow_001 31
flowrepo_covid_EU_030_flow_001 31
flowrepo_covid_EU_031_flow_001 31
flowrepo_covid_EU_032_flow_001 31
flowrepo_covid_EU_033_flow_001 31
flowrepo_covid_EU_034_flow_001 31
flowrepo_covid_EU_035_flow_001 31
flowrepo_covid_EU_036_flow_001 31
flowrepo_covid

In [13]:
for case_id in normalized_data_dict.keys():
    print(case_id)
    for feature, value in normalized_data_dict[case_id].items():
        print(f"  - {feature}: {len(value)}")

flowrepo_covid_EU_002_flow_001
  - SSC-W: 363314
  - SSC-A: 363314
  - SSC-H: 363314
  - FSC-W: 363314
  - FSC-A: 363314
  - FSC-H: 363314
  - FJComp-BUV737-A: 363314
  - FJComp-APC-A: 363314
  - FJComp-BV711-A: 363314
  - FJComp-BB700-P-A: 363314
  - FJComp-BB630-A: 363314
  - FJComp-BUV395-A: 363314
  - FJComp-BUV563-A: 363314
  - FJComp-BV480-A: 363314
  - FJComp-BV421-A: 363314
  - FJComp-BV650-A: 363314
  - FJComp-BYG584-A: 363314
  - FJComp-PE-CF594-A: 363314
  - FJComp-BUV615-P-A: 363314
  - FJComp-BUV805-A: 363314
  - FJComp-BYG790-A: 363314
  - FJComp-PE-Cy5.5-A: 363314
  - FJComp-BV570-A: 363314
  - FJComp-BUV496-A: 363314
  - FJComp-BV605-A: 363314
  - FJComp-BV786-A: 363314
  - FJComp-APC-R700-A: 363314
  - FJComp-BYG670-A: 363314
  - FJComp-BV750-P-A: 363314
  - FJComp-FITC-A: 363314
  - FJComp-APC-H7-A: 363314
flowrepo_covid_EU_003_flow_001
  - SSC-W: 311492
  - SSC-A: 311492
  - SSC-H: 311492
  - FSC-W: 311492
  - FSC-A: 311492
  - FSC-H: 311492
  - FJComp-BUV737-A: 3114

In [14]:
stat_dict = {}
for sample_id, sample_data in normalized_data_dict.items():
    stat_dict[sample_id] = {}
    for channel, values in sample_data.items():
        stat_dict[sample_id][channel + '_mean'] = np.mean(values)
        stat_dict[sample_id][channel + '_std'] = np.std(values)
        stat_dict[sample_id][channel + '_min'] = np.min(values)
        stat_dict[sample_id][channel + '_max'] = np.max(values)

In [24]:
# X = [np.concatenate([data_dict[case_id][channel] for channel in use_channels]) for case_id in sorted(data_dict.keys())]
# y = [labels_dict[case_id] for case_id in sorted(data_dict.keys())]
X = np.array([list(stat_dict[case_id].values()) for case_id in sorted(normalized_data_dict.keys())])
y = np.array([labels_dict[case_id] for case_id in sorted(labels_dict.keys())])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [28]:
model = xgb.XGBClassifier(objective='binary:logistic')

model.fit(X_train, y_train)

y_pred_train = model.predict(X_train)

y_pred_test = model.predict(X_test)

print("Training set predictions:")
print(classification_report(y_train, y_pred_train))

print("Test set predictions:")
print(classification_report(y_test, y_pred_test))

print("Training set confusion matrix:")
print(confusion_matrix(y_train, y_pred_train))

print("Test set confusion matrix:")
print(confusion_matrix(y_test, y_pred_test))

Training set predictions:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        11
           1       1.00      1.00      1.00        21

    accuracy                           1.00        32
   macro avg       1.00      1.00      1.00        32
weighted avg       1.00      1.00      1.00        32

Test set predictions:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.86      0.86      0.86         7

    accuracy                           0.75         8
   macro avg       0.43      0.43      0.43         8
weighted avg       0.75      0.75      0.75         8

Training set confusion matrix:
[[11  0]
 [ 0 21]]
Test set confusion matrix:
[[0 1]
 [1 6]]


In [26]:
clf = svm.SVC(kernel='linear')  
clf.fit(X_train, y_train)
y_pred_train = clf.predict(X_train)
y_pred_test = clf.predict(X_test)

print("Training set predictions:")
print(classification_report(y_train, y_pred_train))

print("Test set predictions:")
print(classification_report(y_test, y_pred_test))

print("Training set confusion matrix:")
print(confusion_matrix(y_train, y_pred_train))

print("Test set confusion matrix:")
print(confusion_matrix(y_test, y_pred_test))

Training set predictions:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        11
           1       1.00      1.00      1.00        21

    accuracy                           1.00        32
   macro avg       1.00      1.00      1.00        32
weighted avg       1.00      1.00      1.00        32

Test set predictions:
              precision    recall  f1-score   support

           0       0.33      1.00      0.50         1
           1       1.00      0.71      0.83         7

    accuracy                           0.75         8
   macro avg       0.67      0.86      0.67         8
weighted avg       0.92      0.75      0.79         8

Training set confusion matrix:
[[11  0]
 [ 0 21]]
Test set confusion matrix:
[[1 0]
 [2 5]]


In [27]:
data.shape

(363314, 35)

## Bonus Question
there are four suggestions:
1. **Clustering**: Clustering is an unsupervised learning technique that can be used to identify and group together data points (cells in this case) with similar characteristics. For example, k-means clustering or DBSCAN could be used to identify the high-density regions of cells. However, these methods would require the optimal number of clusters to be determined.

2. **Anomaly detection**: The yellow cells could be seen as anomalies compared to the rest of the cells. Methods like One-class SVM, Isolation Forest, or Local Outlier Factor can be used to identify these cells as outliers.

3. **Image processing techniques**: Use some image processing techniques such as segmentation and thresholding to identify the high-density regions of cells.

4. **Deep learning**: If have labelled data, a convolutional neural network (CNN) can be used to identify these cells. This would require a labeled dataset for training, where each cell is marked as either 'yellow' or 'not yellow'. The CNN would learn the patterns of the yellow cells and would be able to classify new cells based on this learning.